## Deepwalk 笔记
——首次在网络分析中引入深度学习
>author: xumayi@m.scnu.edu.cn

### Reference
[1] [Perozzi B, Al-Rfou R, Skiena S. Deepwalk: Online learning of social representations[C]//Proceedings of the 20th ACM SIGKDD international conference on Knowledge discovery and data mining. 2014: 701-710.](https://arxiv.org/pdf/1403.6652.pdf)

[2] [Barabási A L, Albert R. Emergence of scaling
in random networks[J]. science, 1999, 286(5439): 509-512.](https://arxiv.org/pdf/cond-mat/9910332.pdf)

[3] [Albert R, Barabási A L. Statistical mechanics of complex networks[J]. Reviews of modern physics, 2002, 74(1): 47.](https://journals.aps.org/rmp/abstract/10.1103/RevModPhys.74.47
)

[4] [Wikipedia: Power law](https://en.wikipedia.org/wiki/Power_law)

[5] [复杂网络的一些相关概念:power law(幂率分布)以及 scale free（无标度）](http://blog.sina.com.cn/s/blog_a5527bf301016qi1.html)

[6] [Python数据可视化系列之幂律分布](https://blog.csdn.net/PythonKiki/article/details/115494380?utm_medium=distribute.pc_relevant.none-task-blog-2~default~baidujs_title~default-0.no_search_link&spm=1001.2101.3001.4242.0)


### 问题定义
对social network 中的节点进行分类

对网络$G=(V,E)$,$V$代表节点，$E$代表连接，$E \subseteq(V \times V)$。

 $G_L=(V,E,X,Y)$是部分被标记的sicial network。

attribute$X \in \mathbb{R}^{|V| \times S}$,$S$是每一个attribute向量的特征空间的维度

$Y \in \mathbb{R}^{|V| \times|\mathcal{Y}|}$,$\mathcal{Y}$是标签的类别数量

传统的分类任务目标：$H(X) \rightarrow \mathcal{Y} $,找到一个合适的$H$

我们的目标：学习$X_E \in \mathbb{R}^{|V| \times d}$，其中$d$是隐藏层的维度，通常较小。

###  图表示与语言模型

Random Walks 的节点路径可以看成是句子。

<img src = https://s3.bmp.ovh/imgs/2021/10/4ab7c46432ca2b22.png width='600'>

如上图所示，（a）来自对图上的一系列的short random walks的统计，（b）来自对
100,000篇来自wikipedia的文章中的文本的统计。显然，这两者都符合**幂次定
律（power law）**。源论文中并未对这一部分进行深层阐述，但个人认为这一发现是
这篇论文最大的贡献，因此进一步

**一些重要概念的补充：**
* power law：幂律分布就是概率密度函数服从幂函数的分布，$y=cx^{-a}$, $(c>0,a>0)$

* zipf定律 : Zipf是哈佛大学的语言学家，他在1932年研究英文单词的出现频率时，发现如果把单词频率
从高到低排列，每个单词的出现频率和它的排名之间存在简单的反比关系：$p(k)= \frac{C}{r^\alpha}$,
经过对数变换：$logP(k)=logC-\alpha logr$。$logC$为常数，令$logC=b$，以$logr$为横坐标，以$logP(k)$为
纵坐标，有上图中所示的坐标系，令$logr=x,logP(k)=y$,
有$y=b-\alpha x$。从（2a）和（2b）中可以看出，当$r$较小时，即network中的vertex访问总数或文本中的单词被使用的总数较少时，其对应的vetrices或单词的量
较大，而当$r$较大时，即network中的vertex访问总数或文本中的单词被使用的总数较大时，其对应的vetrices或单词的量
较小。基本规律总结起来就是：**只有少数英文单词才会被经常使用，大部分的单词很少被使用。同时也有network中，只有少数vertices会被经常访问，而大部分vertices很少被访问。** 这个规律在其他领域的数据分布里也得到了验证，
例如：网页访问频率，
城市人口，
人口收入，
地震震级，
固体破碎时的碎片大小。

* Scala free network(无标度网络):一个网络的度（degree）分布如果服从power law, 则这个网络就叫无标度网络（Scala free network）。
直观的看，直观的看，就是少部分节点度极大，与许多节点相连，而大部分节点度都比较小。许多现实中的网络包括WWW，社交网络，
PPI网络等都被认为是无标度网络，并且大部分实际网络中power law的指数$a$一般都在2和3之间，
并且网络直径相对极小(一般小于$lnN$,$N$为节点数)。

## 算法
主要包含两个主要部分：
* a random walk generator
* a update procedure

\begin{array}{l}
\hline \text { Algorithm 1 DEEPWALK }(G, w, d, \gamma, t) \\
\hline \text { Input: graph } G(V, E) \\
\text { window size } w \\
\text { embedding size } d \\
\text { walks per vertex } \gamma\\
\text { walk length } t \\
\text { Output: matrix of vertex representations } \Phi \in \mathbb{R}^{|V| \times d} \\
\text { 1: Initialization: Sample } \Phi \text { from } \mathcal{U}^{|V| \times d} \\
\text { 2: Build a binary Tree } T \text { from } V \\
\text { 3: for } i=0 \text { to } \gamma \text { do } \\
\text { 4: } \quad \mathcal{O}=\text { Shuffle }(V) \\
\text { 5: } \quad \text { for each } v_{i} \in \mathcal{O} \text { do } \\
\text { 6: } \quad \quad \mathcal{W}_{v_{i}}=\text { RandomWalk }\left(G, v_{i}, \mathrm{t}\right) \\
\text { 7: } \quad \quad \operatorname{Skip-kram}\left(\Phi, \mathcal{W}_{v_{i}}, w\right) \\
\text { 8: } \quad \text{end for}\\
\text { 9: end for } \\
\hline
\end{array}

$\gamma:$在每一个vertex上，以该顶点为起始点进行sample的次数

$t:$每次sample路径的截止长度

$d:\Phi$中每个节点对应的向量表示的维度




第一步：随机采样一个顶点$v_i$作为随机游走路径$\mathcal{W}_{v_{i}}$起始顶点

#### 1）Random Walks
$W_{v_i}:$ 以$v_i$为root的随机路径，$W_{v_i}=\{ \mathcal{W}_{v_i}^1, \mathcal{W}_{v_i}^2,..., \mathcal{W}_{v_i}^k\}$,

#### 2) Language Modeling
给定一个单词序列$W_1^n=(w_0,w_1,...,w_n)$, $w_i \in \mathcal{V}$, ($\mathcal{V}$是词典的大小)

语言模型的优化目标就是使$\operatorname{P}(w_n|w_0,w_1,...,w_{i-1})$最大化

同样，在随机路径中，在给定路径$(v_1,v_2,...,v_{i-1})$的情况下去预测下一个节点$v_{i}$的概率：$\operatorname{P}(v_i|(v_1,v_2,...,v_{i-1}))$。
优化目标即使这个条件概率最大化，同时，我们的最终目的是学习到latent representation，即每个节点的**向量表示**。
在实际计算中，需要将节点转换为可用向量才能进行运算，因此在这里映入一个映射函数$\Phi: v \in V \mapsto \mathbb{R}^{|V| \times d}$,
实际上该文用一个向量矩阵$X_E$来表示$\Phi$。

经过映射函数后，优化问题就变为$\operatorname{P}\left(v_{i} \mid\left(\Phi\left(v_{1}\right), \Phi\left(v_{2}\right), \cdots, \Phi\left(v_{i-1}\right)\right)\right)$。
在实际计算中，这种方法由于计算量巨大，通常不可行。因此，该论文采用使用一个单词来预测上下文单词的方法来替代使用上下文来预测一个缺失的单词的方法。
（即Word2Vec中的Skip-gram的方法）

即优化目标变为$\underset{\Phi}{\operatorname{minimize}}-\log \operatorname{P}\left(\left\{v_{i-w}, \cdots, v_{i+w}\right\} \backslash v_{i} \mid \Phi\left(v_{i}\right)\right)$(如何推导到这一部分参考Word2Vec中Skip-gram的loss函数的计算)


$\operatorname{P}\left(\left\{v_{i-w}, \cdots, v_{i+w}\right\} \backslash v_{i} \mid \Phi\left(v_{i}\right)\right)=\displaystyle\prod_{j=i-w \atop j \neq i}^{i+w} \operatorname{P}\left(v_{j} \mid \Phi\left(v_{i}\right)\right)$

